In [ ]:
!pip install webdriver_manager

In [ ]:
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 92.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 38.6 MB/s eta 0:00:00


In [ ]:
from bs4 import BeautifulSoup
import requests
import csv
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

def get_faculty_info(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'lxml')
    view_content_div = soup.find(class_='view-content')
    faculty_info = view_content_div.find_all(class_='views-field-field-computed-prof-title')
    elements = view_content_div.find_all('h2')
    all_hyperlinks = view_content_div.find_all('a', href=True)
    emails = []
    for link in all_hyperlinks:
        if 'mailto:' in link['href']:
            emails.append(link['href'].split(':')[1])

    #research_area = view_content_div.find_all(class_='views-field-field-research-areas')
    research_areas = []
    phone_numbers = []
    offices = []
    for element in faculty_info:
        research_area = element.find_next_sibling(class_='views-field-field-research-areas')
        phone_number = element.find_next_sibling(class_='views-field views-field-field-computed-phone')
        office = element.find_next_sibling(class_='views-field views-field-field-computed-building')
        if research_area:
            r = research_area.get_text().strip()
            if r.startswith('Research Areas:'):
                r = r.replace("Research Areas:", "").strip()
            research_areas.append(r)
        else:
            research_areas.append("")
        if phone_number:
            p = phone_number.get_text().strip()
            if p.startswith('Phone:'):
                p = p.replace("Phone:", "").strip()
            phone_numbers.append(p)
        else:
            phone_numbers.append("")
        if office:
            office_text = office.get_text().strip()
            if office_text.startswith("Office:"):
                office_text = office_text.replace("Office:", "").strip()
            offices.append(office_text)
        else:
            offices.append("")


    '''for i in range(len(elements)):
        faculty_name = elements[i].get_text()
        faculty_position = faculty_info[i].get_text().strip()
        faculty_email = emails[i]
        print(faculty_name)
        print(faculty_position)
        print(faculty_email)
        print(research_areas[i])
        print(offices[i])
        print(phone_numbers[i])
        print()'''

    with open('research_staff_info.csv', mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Name', 'Position', 'Email', 'Research Area', 'Office', 'Phone Number'])
        for i in range(len(elements)):
            faculty_name = elements[i].get_text()
            faculty_position = faculty_info[i].get_text().strip()
            faculty_email = emails[i]
            writer.writerow([faculty_name, faculty_position, faculty_email, research_areas[i], offices[i], phone_numbers[i]])

def get_text(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'lxml')
    text = soup.get_text()
    with open('kiltie_band.txt', 'w', encoding='utf-8') as file:
        file.write(text)


def get_tables(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'lxml')
    tables = soup.find_all('table')

    for index, table in enumerate(tables, 1):
        filename = f"table_{index}.csv"
        with open(filename, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            rows = table.find_all('tr')
            for row in rows:
                columns = row.find_all(['td', 'th'])
                row_data = [column.get_text().strip() for column in columns]
                writer.writerow(row_data)
        print(f"Table {index} content has been written to {filename} successfully.")


url_faculty_1 = 'https://lti.cs.cmu.edu/directory/all/154/1'
url_faculty_2 = 'https://lti.cs.cmu.edu/directory/all/154/1?page=1'
url_affiliated_faculty = 'https://lti.cs.cmu.edu/directory/all/154/2728'
url_adjunct_faculty = 'https://lti.cs.cmu.edu/directory/all/154/200'
lti_staff_1 = 'https://lti.cs.cmu.edu/directory/all/154/2'
lti_staff_2 = 'https://lti.cs.cmu.edu/directory/all/154/2?page=1'
lti_admin_staff = 'https://lti.cs.cmu.edu/directory/all/154/2731'
lti_research_staff = 'https://lti.cs.cmu.edu/directory/all/154/2730'
spring_carnival = 'https://web.cvent.com/event/ab7f7aba-4e7c-4637-a1fc-dd1f608702c4/websitePage:645d57e4-75eb-4769-b2c0-f201a0bfc6ce?locale=en' #bad website provided
programs = 'https://lti.cs.cmu.edu/learn'
spring_carnival_2 = 'https://www.cmu.edu/engage/alumni/events/campus/spring-carnival/schedule/index.html'
commencement = 'https://www.cmu.edu/commencement/schedule/index.html'
twentyfive_great_things = 'https://www.cs.cmu.edu/scs25/25things'
history = 'https://www.cs.cmu.edu/scs25/history'
history_2 = 'https://www.cmu.edu/about/history.html'
buggy = 'https://www.cmu.edu/news/stories/archives/2019/april/spring-carnival-buggy.html'
tartan_facts = 'https://athletics.cmu.edu/athletics/tartanfacts' #TODO: 403 Error - Fix this
scotty = 'https://athletics.cmu.edu/athletics/mascot/about'
kiltie_band = 'https://athletics.cmu.edu/athletics/kiltieband/index'

#get_faculty_info(lti_research_staff)
#get_tables(programs)
get_text(kiltie_band)


In [ ]:
from bs4 import BeautifulSoup
import requests
import csv

def get_workers_info(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'lxml')
    #print(soup.prettify())
    view_content_div = soup.find(class_='view-content')
    elements = view_content_div.find_all('h2')
    workers_names = [worker_name.get_text() for worker_name in elements]

    return workers_names

'''def schedule_of_classes(url):
    response = requests.get(url)

    if response.status_code == 200:
        html_content= response.text
        soup = BeautifulSoup(html_content, 'html.parser')

        rows = soup.find_all('tr')

        with open('Schedule_of_classes.csv', 'w', newline='', encoding='utf-8') as csvfile:
            csv_writer = csv.writer(csvfile)

            for row in rows:
                columns = row.find_all('td')
                row_data = [column.get_text(strip=True) for column in columns]
                csv_writer.writerow(row_data)

    else:
        print(f"Failed to retrieve the webpage. Status code: {response.status_code}")'''







#all urls
url_faculty = 'https://lti.cs.cmu.edu/directory/all/154/1'
url_affiliated_faculty = 'https://lti.cs.cmu.edu/directory/all/154/2728'
#url_adjunct_faculty = 'https://lti.cs.cmu.edu/directory/all/154/200'
#url_schedule_of_classes = 'https://enr-apps.as.cmu.edu/assets/SOC/sched_layout_spring.htm'


#This will make a csv file of with the timetable of the classes

#TODO
#this is only for the spring semester. I need FALL, summer data too inside my csv file.
#schedule_of_classes(url_schedule_of_classes)




In [ ]:
!pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 97.7 MB/s eta 0:00:00


In [ ]:
!pip install get_data

ERROR: Could not find a version that satisfies the requirement get_data (from versions: none)
ERROR: No matching distribution found for get_data


In [ ]:
!pip install semanticscholar

In [ ]:
from semanticscholar import SemanticScholar
from bs4 import BeautifulSoup
import requests
import json
#from get_data import *
import pandas as pd
import time

def research_papers(faculty_name):
    url = "https://api.semanticscholar.org/graph/v1/author/search"
    authors_name = faculty_name

    query_params = {
        "query": authors_name,
        'year': '2023',
        "fields": "paperCount,papers.title,papers.abstract,papers.openAccessPdf,papers.authors,papers.year,papers.publicationVenue,papers.isOpenAccess"
    }


    response = requests.get(url, params=query_params)

    if response.status_code == 200:
        #time.sleep(1)
        data = response.json()
        all_authors_data[faculty_name] = data

        papers = data.get('papers', [])


    else:
        print(f"Request failed with status code {response.status_code}")


def summary_research_papers(json_file_name, text_file_path):
    json_url = json_file_name
    with open(json_url, "r") as file:
        json_data = json.load(file)

    file_path = text_file_path
    with open(file_path, "w+", encoding='utf-8') as file:
        paper_number = 0
        for author, research_paper_data in json_data.items():
            for paper_data in research_paper_data["data"]:
                for data in paper_data["papers"]:
                    if data["year"] == 2023:
                        faculty_name = author
                        faculty_author_id = paper_data["authorId"]
                        paper_title = data["title"]
                        if data["isOpenAccess"] == True:
                            open_access_bool = data["isOpenAccess"]
                            publication_link = data["openAccessPdf"]
                            year_published = data["year"]
                            paper_abstract = data["abstract"]
                            paper_id = data["paperId"]
                            if publication_link is None:
                                publication_link = 'Its not open access'
                                url ='Not given'
                            else:
                                url = publication_link['url']

                            r = requests.post('https://api.semanticscholar.org/graph/v1/paper/batch', params={'fields': 'tldr'}, json={"ids": [paper_id]})
                            data = r.json()
                            if data and isinstance(data, list) and len(data) > 0:
                                tldr_info = data[0].get('tldr', {})
                                if tldr_info == None:
                                    tldr_text = ''
                                else:
                                    tldr_text = tldr_info.get('text', '')

                            file.write(f"faculty_name: {faculty_name}\nfaculty_authorid: {faculty_author_id}\npaper_id: {paper_id}\npaper_title: {paper_title}\npublication_link: {url} \nyear_published: {year_published} \nabstract_paper:{paper_abstract}\nisOpenAccess: {open_access_bool}\nTL\DR: {tldr_text}\n================================\n")



def paper_pdf_links(json_file_name, file_path):
    json_url = json_file_name
    with open(json_url, "r") as file:
        json_data = json.load(file)

    with open(file_path, "w+", encoding='utf-8') as file:
        for author, research_paper_data in json_data.items():
            for paper_data in research_paper_data["data"]:
                for x in paper_data["papers"]:
                    if x["year"] == 2023:
                        publication_link = x["openAccessPdf"]
                        if publication_link is None:
                            publication_link = 'Its not open access'
                            url ='Not given'
                        else:
                            url = publication_link['url']
                            file.write(f"{url}\n")



all_authors_data = {}

url_faculty = 'https://web.archive.org/web/20231130214911/https://lti.cs.cmu.edu/directory/all/154/1'
url_faculty_page2 = 'https://web.archive.org/web/20231209211749/https://lti.cs.cmu.edu/directory/all/154/1?page=1'

faculty_names = get_workers_info(url_faculty)
faculty_names = get_workers_info(url_faculty_page2)+faculty_names

file_path = '/content/sample_data/faculty_info.csv'
file_path_2 = '/content/sample_data/faculty_info_2.csv'
df = pd.read_csv(file_path)
df2 = pd.read_csv(file_path_2)
col_name = 'Name'
faculty_name1 = df[col_name].tolist()
faculty_name2 = df2[col_name].tolist()
faculty_names = faculty_name1 + faculty_name2
print((faculty_name1[1]))

for faculty in faculty_names:
    print(faculty)
    research_papers(f'{faculty}')



#research_papers('Graham Neubig')

print('Faculty data pulling done')
json_file_name = 'json_file_faculties/grahamneubig.json'
text_file_path = 'metadata/grahamneubig'
filepath = 'paperlinks/grahamneubig'

with open(json_file_name, "w") as file:
    json.dump(all_authors_data, file, indent=4)

print('Created the JSON file')

summary_research_papers(json_file_name, text_file_path)
print('meta data created')

paper_pdf_links(json_file_name, filepath)
print('all research papers generated')


Ralf Brown
Yonatan Bisk
Ralf Brown
Jamie Callan
Justine Cassell
Mona Diab
Fernando Diaz
Scott Fahlman
Robert Frederking
Daniel Fried
Anatole Gershman
Alexander Hauptmann
Daphne Ippolito
Lori Levin
Lei Li
Teruko Mitamura
Louis-Philippe Morency
David Mortensen
Graham Neubig
Eric Nyberg
Kemal Oflazer
Bhiksha Ramakrishnan
Carolyn Rosé
Alexander Rudnicky
Maarten Sap
Michael Shamos
Rita Singh
Emma Strubell
Alexander Waibel
Shinji Watanabe
Sean Welleck
Eric P. Xing
Chenyan Xiong
Yiming Yang
Faculty data pulling done


FileNotFoundError: [Errno 2] No such file or directory: 'json_file_faculties/grahamneubig.json'

In [ ]:
urls = [
    "https://en.wikipedia.org/wiki/Pittsburgh",
    "https://en.wikipedia.org/wiki/History_of_Pittsburgh",
    "https://www.pittsburghpa.gov/",
    "https://www.britannica.com/place/Pittsburgh",
    "https://pittsburghpa.gov/events",
    "https://pittsburghsymphony.org/",
    "https://pittsburgh.events/",
    "https://downtownpittsburgh.com/events/",
    "https://www.pghcitypaper.com/pittsburgh/EventSearch?v=d",
    "https://events.cmu.edu/",
    "https://www.cmu.edu/engage/alumni/events/campus/index.html",
    "https://pittsburghopera.org/",
    "https://trustarts.org/",
    "https://carnegiemuseums.org/",
    "https://www.heinzhistorycenter.org/",
    "https://www.thefrickpittsburgh.org/",
    "https://en.wikipedia.org/wiki/List_of_museums_in_Pittsburgh",
    "https://www.visitpittsburgh.com/events-festivals/food-festivals/",
    "https://www.picklesburgh.com/",
    "https://www.pghtacofest.com/",
    "https://pittsburghrestaurantweek.com/",
    "https://littleitalydays.com/",
    "https://bananasplitfest.com/",
    "https://www.visitpittsburgh.com/things-to-do/pittsburgh-sports-teams/",
    "https://www.mlb.com/pirates",
    "https://www.steelers.com/",
    "https://www.nhl.com/penguins/",
    # ... thêm các URL khác từ danh sách trong ảnh
]


In [ ]:
import requests
from bs4 import BeautifulSoup

def crawl_page(url):
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        # Lấy tất cả các đoạn văn, tiêu đề, và danh sách (và có thể điều chỉnh thêm)
        paragraphs = soup.find_all('p')
        headers = soup.find_all(['h1', 'h2', 'h3'])
        lists = soup.find_all('ul')

        # Kết hợp nội dung từ các đoạn văn, tiêu đề và danh sách
        content = "\n".join(p.get_text() for p in paragraphs if p.get_text().strip())
        content += "\n".join(h.get_text() for h in headers if h.get_text().strip())
        content += "\n".join(li.get_text() for li in lists if li.get_text().strip())

        return content[:100000]  # Tăng độ dài nội dung nếu cần
    except Exception as e:
        return f"Error fetching {url}: {e}"

with open("crawled_pages.txt", "w", encoding="utf-8") as f:
    for url in urls:
        print(f"--- Crawling: {url} ---")
        content = crawl_page(url)
        # Ghi vào file chỉ nội dung, không URL
        f.write(content)
        f.write("\n" + "="*80 + "\n\n")
        # In trước 500 ký tự đầu tiên để xem
        print(content[:500])
        print("\n" + "="*80 + "\n")

--- Crawling: https://en.wikipedia.org/wiki/Pittsburgh ---
Pittsburgh (/ˈpɪtsbɜːrɡ/ PITS-burg) is a city in Allegheny County, Pennsylvania, United States, and its county seat. It is the second-most populous city in Pennsylvania (after Philadelphia) and the 68th-most populous city in the U.S., with a population of 302,971 as of the 2020 census. The city is located in southwestern Pennsylvania at the confluence of the Allegheny River and Monongahela River, which combine to form the Ohio River.[7] It anchors the Pittsburgh metropolitan area, which had a po


--- Crawling: https://en.wikipedia.org/wiki/History_of_Pittsburgh ---
The history of Pittsburgh began with centuries of Native American civilization in the modern Pittsburgh region, known as Jaödeogë’ in the Seneca language.[1] Eventually, European explorers encountered the strategic confluence where the Allegheny and Monongahela Rivers meet to form the Ohio, which leads to the Mississippi River. The area became a battleground when Fr

In [ ]:
!pip install pypdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.4/303.4 kB 11.3 MB/s eta 0:00:00


In [ ]:
'''
Script for pdf crawling and text extraction
'''

import os
from pypdf import PdfReader

def extract_text_from_pdf(pdf_path):
    # Create a PDF reader object
    reader = PdfReader(pdf_path)

    # Initialize a string to hold the extracted text
    extracted_text = ""

    # Loop through each page in the PDF
    for page_num in range(len(reader.pages)):
        page = reader.pages[page_num]
        extracted_text += page.extract_text() + "\n"  # Extract text from each page and add newline for separation

    return extracted_text

# Function to list all PDF files in a directory
def list_pdf_files(directory):
    return [f for f in os.listdir(directory) if f.endswith('.pdf')]

if __name__ == "__main__":
    # Directory containing the PDF files
    pdf_file_dir = '../../data/raw/raw_pdf_data'

    # Get all PDF files in the directory
    pdf_files = list_pdf_files(pdf_file_dir)

    # Process each PDF file
    for pdf_file in pdf_files:
        pdf_file_path = os.path.join(pdf_file_dir, pdf_file)  # Full path to the PDF file
        print(f"Processing {pdf_file}...")

        # Extract text from the PDF
        text = extract_text_from_pdf(pdf_file_path)

        # Create a corresponding text file name (same name as the PDF, but with .txt extension)
        txt_file_name = os.path.splitext(pdf_file)[0] + ".txt"
        txt_file_path = os.path.join('../../data/crawled/crawled_pdf_text_data', txt_file_name)

        # Save the extracted text to a text file
        with open(txt_file_path, 'w', encoding='utf-8') as text_file:
            cleaned_text = text.replace('\n', ' ')
            text_file.write(cleaned_text)

        print(f"Saved extracted text to {txt_file_name}")

FileNotFoundError: [Errno 2] No such file or directory: '../../data/raw/raw_pdf_data'

In [ ]:
!pip install requests beautifulsoup4


In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

# Hàm crawl PDF từ 1 URL
def find_pdf_links_from_url(base_url):
    try:
        response = requests.get(base_url, headers={'User-Agent': 'Mozilla/5.0'})
        soup = BeautifulSoup(response.text, 'html.parser')
        pdf_links = []

        for link in soup.find_all('a', href=True):
            href = link['href']
            if href.endswith('.pdf'):
                full_url = urljoin(base_url, href)  # nối nếu link tương đối
                pdf_links.append(full_url)

        return pdf_links
    except Exception as e:
        print(f"Lỗi với {base_url}: {e}")
        return []

# Danh sách URL cần crawl
url_list = [
    "https://en.wikipedia.org/wiki/Pittsburgh",
    "https://en.wikipedia.org/wiki/History_of_Pittsburgh",
    "https://www.pittsburghpa.gov/",
    "https://www.britannica.com/place/Pittsburgh",
    "https://pittsburghpa.gov/events",
    "https://pittsburghsymphony.org/",
    "https://pittsburgh.events/",
    "https://downtownpittsburgh.com/events/",
    "https://www.pghcitypaper.com/pittsburgh/EventSearch?v=d",
    "https://events.cmu.edu/",
    "https://www.cmu.edu/engage/alumni/events/campus/index.html",
    "https://pittsburghopera.org/",
    "https://trustarts.org/",
    "https://carnegiemuseums.org/",
    "https://www.heinzhistorycenter.org/",
    "https://www.thefrickpittsburgh.org/",
    "https://en.wikipedia.org/wiki/List_of_museums_in_Pittsburgh",
    "https://www.visitpittsburgh.com/events-festivals/food-festivals/",
    "https://www.picklesburgh.com/",
    "https://www.pghtacofest.com/",
    "https://pittsburghrestaurantweek.com/",
    "https://littleitalydays.com/",
    "https://bananasplitfest.com/",
    "https://www.visitpittsburgh.com/things-to-do/pittsburgh-sports-teams/",
    "https://www.mlb.com/pirates",
    "https://www.steelers.com/",
    "https://www.nhl.com/penguins/",
    # ... thêm các URL khác từ danh sách trong ảnh
]

# Tìm tất cả link PDF từ các URL
all_pdf_links = []

for url in url_list:
    print(f"🔎 Đang crawl: {url}")
    pdfs = find_pdf_links_from_url(url)
    all_pdf_links.extend(pdfs)
    print(f"✅ Tìm thấy {len(pdfs)} link PDF\n")

# In kết quả
print("📄 TỔNG SỐ LINK PDF:", len(all_pdf_links))
for link in all_pdf_links:
    print(link)


🔎 Đang crawl: https://en.wikipedia.org/wiki/Pittsburgh
✅ Tìm thấy 17 link PDF

🔎 Đang crawl: https://en.wikipedia.org/wiki/History_of_Pittsburgh
✅ Tìm thấy 2 link PDF

🔎 Đang crawl: https://www.pittsburghpa.gov/
✅ Tìm thấy 0 link PDF

🔎 Đang crawl: https://www.britannica.com/place/Pittsburgh
✅ Tìm thấy 0 link PDF

🔎 Đang crawl: https://pittsburghpa.gov/events
✅ Tìm thấy 0 link PDF

🔎 Đang crawl: https://pittsburghsymphony.org/
✅ Tìm thấy 0 link PDF

🔎 Đang crawl: https://pittsburgh.events/
✅ Tìm thấy 0 link PDF

🔎 Đang crawl: https://downtownpittsburgh.com/events/
✅ Tìm thấy 0 link PDF

🔎 Đang crawl: https://www.pghcitypaper.com/pittsburgh/EventSearch?v=d
✅ Tìm thấy 0 link PDF

🔎 Đang crawl: https://events.cmu.edu/
✅ Tìm thấy 0 link PDF

🔎 Đang crawl: https://www.cmu.edu/engage/alumni/events/campus/index.html
✅ Tìm thấy 0 link PDF

🔎 Đang crawl: https://pittsburghopera.org/
✅ Tìm thấy 0 link PDF

🔎 Đang crawl: https://trustarts.org/
✅ Tìm thấy 0 link PDF

🔎 Đang crawl: https://carnegie

In [ ]:
!pip install pypdf requests


In [ ]:
import requests
from pypdf import PdfReader
import io
import os

# Thư mục lưu
os.makedirs("downloaded_pdfs", exist_ok=True)
os.makedirs("extracted_texts", exist_ok=True)

# 1. Danh sách các URL PDF
pdf_urls = [
    "http://www.nsf.gov/statistics/infbrief/nsf13305/nsf13305.pdf",
    "http://174.143.38.57/wp-content/uploads/2010/06/S013_ROBOT-RxSellSheet.pdf",
    "http://www.co.greene.pa.us/secured/gc2/history/Struggle-for-Possession.pdf",
    "https://upittpress.org/wp-content/uploads/2019/01/9780822943716exr.pdf",
    "https://opapgh.org/wp-content/uploads/2020/08/OPA_ArtinPublicPlaces_RetailFirstside.pdf",
    "https://www.weather.gov/media/pbz/records/prec.pdf",
    "https://www2.census.gov/library/publications/decennial/1980/volume-1/pennsylvania/1980censusofpopu80140un_bw.pdf",
    "https://www2.census.gov/library/publications/decennial/1990/cp-1/cp-1-40-1.pdf",
    "https://www.americanimmigrationcouncil.org/sites/default/files/research/council_new_americans_in_pittsburgh_9_2023.pdf",
    "https://web.archive.org/web/20191221235539/https://www.brandeis.edu/ssri/pdfs/communitystudies/PittsburghJewishCommStudy.pdf",
    "https://www.brandeis.edu/ssri/pdfs/communitystudies/PittsburghJewishCommStudy.pdf",
    "https://web.archive.org/web/20140202184957/http://obs.rc.fas.harvard.edu/chetty/mobility_geo.pdf",
    "https://www.ceapittsburgh.org/wp-content/uploads/2020/01/A-Ground-Up-Model-for-Gun-Violence-Reduction.pdf",
    "https://aeg.memberclicks.net/assets/docs/Geology%20of%20Pittsburgh%20Book.pdf",
    "https://web.archive.org/web/20070927220849/http://www.apta.com/research/stats/ridership/riderep/documents/07q1bus.pdf",
    "http://www.apta.com/research/stats/ridership/riderep/documents/07q1bus.pdf",
    "http://aapa.files.cms-plus.com/PDFs/2011%20U%20S%20%20PORT%20RANKINGS%20BY%20CARGO%20TONNAGE.pdf",
    "http://docs.lib.noaa.gov/rescue/mwr/055/mwr-055-11-0500a.pdf",
    "http://littleitalydays.com/wp-content/uploads/2024/07/MISS-LITTLE-ITALY-2024.pdf",
    "http://littleitalydays.com/wp-content/uploads/2019/07/GolffatherEntryForm.pdf",
    "http://littleitalydays.com/wp-content/uploads/2019/07/GolffatherEventOverview.pdf"
]


# 2. Kiểm tra xem file có phải PDF thật không
def is_valid_pdf(file_bytes):
    file_bytes.seek(0)
    return file_bytes.read(4) == b'%PDF'

# 3. Tải file PDF từ URL
def download_pdf_from_url(url):
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            return io.BytesIO(response.content)
        else:
            print(f"❌ Không thể tải: {url} — Mã lỗi: {response.status_code}")
            return None
    except Exception as e:
        print(f"❌ Lỗi khi tải {url}: {e}")
        return None

# 4. Trích xuất text từ PDF
def extract_text_from_pdf(pdf_file):
    reader = PdfReader(pdf_file)
    text = ""
    for page in reader.pages:
        page_text = page.extract_text()
        if page_text:
            text += page_text + "\n"
    return text

# 5. Lặp qua từng URL, tải và trích xuất text
for i, url in enumerate(pdf_urls):
    print(f"\n🔽 Đang xử lý file {i+1}: {url}")
    pdf_file = download_pdf_from_url(url)

    if pdf_file and is_valid_pdf(pdf_file):
        pdf_filename = f"downloaded_pdfs/file_{i+1}.pdf"
        text_filename = f"extracted_texts/file_{i+1}.txt"

        # Lưu file PDF
        with open(pdf_filename, 'wb') as f:
            f.write(pdf_file.getbuffer())
        print(f"✅ Đã lưu file: {pdf_filename}")

        # Trích xuất và lưu text
        text = extract_text_from_pdf(pdf_file)
        with open(text_filename, 'w', encoding='utf-8') as f:
            f.write(text)
        print(f"📝 Đã trích xuất và lưu văn bản vào: {text_filename}")

    else:
        print(f"⚠️ Bỏ qua: File không hợp lệ hoặc không phải PDF: {url}")



🔽 Đang xử lý file 1: http://www.nsf.gov/statistics/infbrief/nsf13305/nsf13305.pdf
⚠️ Bỏ qua: File không hợp lệ hoặc không phải PDF: http://www.nsf.gov/statistics/infbrief/nsf13305/nsf13305.pdf

🔽 Đang xử lý file 2: http://174.143.38.57/wp-content/uploads/2010/06/S013_ROBOT-RxSellSheet.pdf
❌ Lỗi khi tải http://174.143.38.57/wp-content/uploads/2010/06/S013_ROBOT-RxSellSheet.pdf: HTTPConnectionPool(host='174.143.38.57', port=80): Max retries exceeded with url: /wp-content/uploads/2010/06/S013_ROBOT-RxSellSheet.pdf (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x7fe815428ed0>, 'Connection to 174.143.38.57 timed out. (connect timeout=10)'))
⚠️ Bỏ qua: File không hợp lệ hoặc không phải PDF: http://174.143.38.57/wp-content/uploads/2010/06/S013_ROBOT-RxSellSheet.pdf

🔽 Đang xử lý file 3: http://www.co.greene.pa.us/secured/gc2/history/Struggle-for-Possession.pdf
❌ Lỗi khi tải http://www.co.greene.pa.us/secured/gc2/history/Struggle-for-Possession.pdf: HTTPConnectio

In [ ]:
import os

# Thư mục chứa các file .txt đã trích xuất
text_dir = "/content/extracted_texts"
output_file = "all_texts_combined.txt"

# Lấy danh sách tất cả các file .txt trong thư mục
text_files = sorted([f for f in os.listdir(text_dir) if f.endswith('.txt')])

# Gộp nội dung từng file vào file tổng
with open(output_file, 'w', encoding='utf-8') as outfile:
    for text_file in text_files:
        file_path = os.path.join(text_dir, text_file)
        with open(file_path, 'r', encoding='utf-8') as infile:
            content = infile.read()
            outfile.write(f"=== {text_file} ===\n")  # Ghi tiêu đề mỗi file
            outfile.write(content + "\n\n")

print(f"✅ Đã gộp {len(text_files)} file vào: {output_file}")


✅ Đã gộp 8 file vào: all_texts_combined.txt


###Tiền xử lý dữ liệu

In [ ]:
!pip install nltk beautifulsoup4 emoji pyspellchecker


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 78.3 MB/s eta 0:00:00


In [ ]:
import shutil
shutil.rmtree('/root/nltk_data', ignore_errors=True)


In [ ]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
with open('/all_texts_combined.txt', 'r', encoding='utf-8') as f:
    content = f.read()

In [ ]:
import pandas as pd

# Đọc CSV
df = pd.read_csv("/test_questions.csv")

# Lấy cột "question" và ghi từng dòng vào file text
with open("questions.txt", "w", encoding="utf-8") as f:
    for q in df["Question"]:
        f.write(q.strip() + "\n")
